In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
diabetes_data = pd.read_csv(os.path.join(dirname, filename))
print("Check the number of records:  ", diabetes_data.shape)
diabetes_data.head()

In [ ]:
diabetes_data.info(verbose=True)

Data Exploration

In [ ]:
diabetes_data.describe()

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
def plotPie(df, nameOfFeature, labels):
#     labels = [str(df[nameOfFeature].unique()[i]) for i in range(df[nameOfFeature].nunique())]
    values = [df[nameOfFeature].value_counts()[i] for i in range(df[nameOfFeature].nunique())]
    trace=go.Pie(labels=labels,values=values)
    py.iplot([trace])

In [ ]:
df = diabetes_data.copy(deep = True)
df['Outcome'] = df['Outcome'].replace({1: 'Positive', 0: 'Negative'})
# print(df[df['Outcome']=='Positive'].count())
plotPie(df, 'Outcome', ['Negative','Positive'])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(diabetes_data.corr(),annot=True, cmap = 'YlGnBu')
fig = plt.gcf()
fig.set_size_inches(10, 8)
plt.show()

In [ ]:
# diabetes_data_copy = diabetes_data.copy(deep = True)

for col in diabetes_data:
    if(col != 'Pregnancies' and col!='SkinThickness' and col != 'Outcome' and col != 'DiabetesPedigreeFunction'):
        diabetes_data[col] = diabetes_data[col].replace(0,np.NaN)
## showing the count of Nans
print(diabetes_data.isnull().sum())

In [ ]:
for col in diabetes_data:
    diabetes_data[col].fillna(diabetes_data[col].mean(), inplace = True)
print(diabetes_data.isnull().sum())

In [ ]:
sns.heatmap(diabetes_data.corr(),annot=True, cmap = 'YlGnBu')
fig = plt.gcf()
fig.set_size_inches(10, 8)
plt.show()

In [ ]:
plt.figure(figsize=(60,60))
for index, col in enumerate(diabetes_data,1):
    if(col == 'Outcome'):
        continue
    sub_plot_index = (index - 1) *2
    plt.subplot(3,3, index)
    sns.violinplot(x='Outcome', y=f'{col}', data=diabetes_data, palette='muted', split=True)


In [ ]:
X = diabetes_data.drop(['Outcome'],axis = 1)
Y = diabetes_data.Outcome

In [ ]:
#importing train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=42, stratify=Y)

In [ ]:
X_feature_imp = diabetes_data[['Glucose','BMI','Age','Pregnancies']]
Y_feature_imp = diabetes_data.Outcome
X_train_feature_imp,X_test_feature_imp,Y_train_feature_imp,Y_test_feature_imp = train_test_split(X_feature_imp,Y_feature_imp,test_size=0.25,random_state=42, stratify=Y_feature_imp)

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
# def train(X_train, X_test, Y_train, Y_test, model):
from sklearn.metrics import classification_report, confusion_matrix
def train(model):
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    score = metrics.accuracy_score(Y_test, Y_pred)
    print('Score: %0.4f'%score)
#     print(classification_report(model.predict(X_test), Y_test))
    cm = confusion_matrix(Y_test, Y_pred)
    sns.set(color_codes =True)
    sns.set(font_scale=1.5)
    sns.heatmap(cm, annot=True, fmt='g')
    plt.show()
    return model

In [ ]:
# def train(X_train, X_test, Y_train, Y_test, model):
def train_feature_imp(model):
    model.fit(X_train_feature_imp, Y_train_feature_imp)
    Y_pred_feature_imp = model.predict(X_test_feature_imp)
    score = metrics.accuracy_score(Y_test_feature_imp, Y_pred_feature_imp)
    print('Score: %0.4f'%score)
#     print(classification_report(model.predict(X_test_feature_imp), Y_test_feature_imp))
    cm = confusion_matrix(Y_test_feature_imp, Y_pred_feature_imp)
    sns.set(color_codes =True)
    sns.set(font_scale=1.5)
    sns.heatmap(cm, annot=True, fmt='g')
    plt.show()
    return model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
model_LR = LogisticRegression(max_iter=2500)
train(model_LR)
train_feature_imp(model_LR)

In [ ]:
accuracy_rate = []
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
for i in range(1, 30):
    knn = KNeighborsClassifier(n_neighbors=i)
    score = cross_val_score(knn, X, Y, cv=20)
    accuracy_rate.append(score.mean())
accuracy_rate

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(range(1,30),accuracy_rate, color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('accuracy_rate vs K value')
plt.xlabel('K value')
plt.ylabel('accuracy_rate')

In [ ]:

model_knn = KNeighborsClassifier(n_neighbors=20)
train(model_knn)
train_feature_imp(model_knn)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rdc = RandomForestClassifier()
train(model_rdc)
train_feature_imp(model_rdc)